# Import package

In [54]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Data path

In [17]:
root = '/raid/Broden/dataset/broden1_227'
input_dir = os.path.join(root, 'images')
output_dir = os.path.join(root, 'parsed_images/object')

In [7]:
subdirs = ['ade20k', 'dtd', 'opensurfaces', 'pascal']
categories = ['color', 'object', 'part', 'material', 'scene', 'texture']

def include_category_in_name(file, categories):
    for c in categories:
        if c in file:
            return True
    return False

In [21]:
img_names = {}
for subdir in subdirs:
    dirpath = os.path.join(input_dir, subdir)
    files = os.listdir(dirpath)
    for file in files:
        if include_category_in_name(file, categories) and 'object' in file:
            img_name = file[:-11]
            if os.path.exists(os.path.join(dirpath, f'{img_name}.png')):
                img_p = os.path.join(dirpath, f'{img_name}.png')
            elif os.path.exists(os.path.join(dirpath, f'{img_name}.jpg')):
                img_p = os.path.join(dirpath, f'{img_name}.jpg')
            img_names[img_name] = [img_p, os.path.join(dirpath, file)]

In [22]:
img_names['ADE_train_00017899']

['/raid/Broden/dataset/broden1_227/images/ade20k/ADE_train_00017899.jpg',
 '/raid/Broden/dataset/broden1_227/images/ade20k/ADE_train_00017899_object.png']

# Parse image

In [55]:
with tqdm(total=len(img_names)) as pbar:
    for img_name in img_names:
        img_p, obj_img_p = img_names[img_name]
        img = cv2.resize(plt.imread(img_p), (256, 256))
        obj_img = cv2.resize(plt.imread(obj_img_p), (256, 256))

        if np.sum(obj_img) > 0:
            masked_img = np.zeros((256, 256, 3))
            for row in range(256):
                for col in range(256):
                    obj_pixel = obj_img[row, col, :]
                    if np.sum(obj_pixel) > 0.1:
                        masked_img[row, col] = img[row, col]
            p = os.path.join(output_dir, f'{img_name}.jpg')
            plt.imsave(p, masked_img / 255)
        pbar.update(1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32313/32313 [3:58:32<00:00,  2.26it/s]
